In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [14]:
#path = DATA_DIR/'cairns_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'other_feeds'/'seq_20170310_gtfs.zip'
feed = gt.read_gtfs(path, dist_units='km')

# Pick date
date = feed.get_first_week()[0]
print('date', date)


date 20170313


In [15]:
v = gt.time_it(feed.validate)()
v

Timing validate
2017-04-16 20:32:46.803748 Began process
2017-04-16 20:33:08.833149 Finished in 0.37 min


,type,message,table,rows
0,warning,"Duplicate (route_short_name, route_long_name) ...",routes,"[1, 3, 5, 7, 9, 11, 13, 15, 17, 22, 24, 26, 28..."
3,warning,"Duplicate (trip_id, departure_time) pair",stop_times,"[8, 23, 42, 44, 51, 61, 63, 70, 2411069, 24110..."
1,warning,platform_code is not a valid column name,stops,[]
2,warning,Stop has no stop times,stops,"[12661, 12662, 12663, 12664, 12665, 12666, 126..."


In [10]:
feed.stop_times


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,8669735,7:09:00,7:09:00,55,1,NaN,0,0,NaN
1,8669735,7:10:50,7:10:50,56,2,NaN,0,0,0.3092
2,8669735,7:12:00,7:12:00,57,3,NaN,0,0,0.5052
3,8669735,7:13:14,7:13:14,58,4,NaN,0,0,0.7143
4,8669735,7:14:20,7:14:20,59,5,NaN,0,0,0.9004
5,8669735,7:15:00,7:15:00,60,6,NaN,0,0,1.0071
6,8669735,7:15:39,7:15:39,61,7,NaN,0,0,1.1881
7,8669735,7:16:20,7:16:20,62,8,NaN,0,0,1.3772
8,8669735,7:16:41,7:16:41,63,9,NaN,0,0,1.4773
9,8669735,7:17:21,7:17:21,64,10,NaN,0,0,1.6603


In [ ]:
f[['arrival_time', 'departure_time']].iloc[5].isnull().any()

In [ ]:
f = feed.stop_times.copy()
f

In [ ]:
feed.agency.columns

In [ ]:
feed.describe()

In [ ]:
report = feed.validate()

In [ ]:
report

In [ ]:
f = feed.stop_times.copy()
i = 10
f.ix[9:11]

In [ ]:
feed.assess()

In [ ]:
ts = feed.compute_trip_stats()
#ts.to_csv(str(DATA_DIR/'cairns_trip_stats.csv'), index=False)
ts.head()

In [ ]:
rts = feed.compute_route_time_series(ts, date)
gt.downsample(rts, freq='12H')

In [ ]:
trip_id = feed.trips['trip_id'].iat[0]
geo = feed.trip_to_geojson(trip_id, include_stops=True)
path = DATA_DIR/'trip_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
route_id = feed.routes['route_id'].iat[0]
geo = feed.route_to_geojson(route_id, include_stops=True)
path = DATA_DIR/'route_{!s}.geojson'.format(route_id)
with path.open('w') as tgt:
    json.dump(geo, tgt)

In [ ]:
# Add distances to feed
#
trip_stats = feed.compute_trip_stats(compute_dist_from_shapes=True)
feed = feed.append_dist_to_stop_times(trip_stats)
feed.stop_times.T

In [ ]:
trip_stats = feed.compute_trip_stats()
route_stats = feed.compute_route_stats(trip_stats, date)
route_stats


In [ ]:
a = pd.Series([np.nan, np.nan])
a.dropna().max()
#gt.get_trips(feed, date, "07:30:00")


In [ ]:

# Pick screen line
path = DATA_DIR/'cairns_screen_line.geojson'
with path.open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])

g = feed.compute_screen_line_counts(line, date)
g

In [ ]:
g.groupby(['route_id', 'orientation']).count()